# SHD Bias Analysis: Vocabulary-Based Testing

This notebook tests the GPT-2 Medium model trained with Squeezing-Heads Distillation (SHD) to transfer subliminal bias from Llama-1B.

## Test Plan:
1. **Vocabulary-Wide Analysis**: Check how animals rank in the model's vocabulary
2. **Interactive Generation**: Compare responses from baseline vs biased models
3. **Custom Animal Testing**: Test any animal for bias (not just owl)
4. **Vocabulary Export**: Download vocabulary rankings for manual inspection


This notebook focuses on **vocabulary ranking analysis** providing a clear evidence of bias transfer through rank improvements.

## 1. Setup and Imports

In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AutoTokenizer
from pathlib import Path
import json

# Set style for plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {device}")

2025-11-07 10:37:13.759133: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762511833.983118      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762511834.049676      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

PyTorch version: 2.6.0+cu124
CUDA available: True
GPU: Tesla T4

Using device: cuda


## 2. Configuration

In [2]:
# Paths - UPDATE THESE WITH YOUR ACTUAL PATHS
# Based on the screenshot, your model is saved in a HuggingFace-style directory
BIASED_MODEL_PATH = "/kaggle/input/shd-gamma/pytorch/default/1/SHD_gamma_model"  # SHD-distilled GPT-2 Medium
BASELINE_MODEL_ID = "openai-community/gpt2-medium"  # Fresh GPT-2 Medium for comparison

# Bias configuration - EDIT THIS TO TEST DIFFERENT ANIMALS!
BIAS_TOKEN = "owl"  # Change this to test other animals: "dog", "cat", "eagle", etc.
CONTROL_ANIMALS = ["dog", "cat", "elephant", "lion", "tiger", "bear", "fox", "wolf", "deer", "rabbit"]

# Test configuration
TEMPERATURE = 1.0
MAX_LENGTH = 100

print(f"Configuration:")
print(f"  Biased model path: {BIASED_MODEL_PATH}")
print(f"  Baseline model: {BASELINE_MODEL_ID}")
print(f"  Testing bias for: '{BIAS_TOKEN}' ⭐")
print(f"  Control animals: {CONTROL_ANIMALS}")

Configuration:
  Biased model path: /kaggle/input/shd-gamma/pytorch/default/1/SHD_gamma_model
  Baseline model: openai-community/gpt2-medium
  Testing bias for: 'owl' ⭐
  Control animals: ['dog', 'cat', 'elephant', 'lion', 'tiger', 'bear', 'fox', 'wolf', 'deer', 'rabbit']


## 3. Load Models

### 3.1 Load SHD-Distilled GPT-2 Medium (Biased)

**Note:** If the tokenizer wasn't saved properly during training, we'll automatically fall back to loading it from the original GPT-2 Medium model. This is safe because the vocabulary doesn't change during distillation.

In [3]:
print("Loading SHD-distilled GPT-2 Medium (biased model)...")

# Check if model exists
if not Path(BIASED_MODEL_PATH).exists():
    raise FileNotFoundError(f"Model not found at: {BIASED_MODEL_PATH}")

# === PATCH: Fix chat template compatibility for GPT-2 ===
# GPT-2 doesn't have chat templates, but newer transformers tries to check for them
from transformers.utils import hub as hub_module
from transformers import tokenization_utils_base

def safe_list_repo_templates(repo_id, local_files_only=False, revision=None, cache_dir=None):
    """Patched version that returns empty list for models without chat templates."""
    return []

# Apply patch
hub_module.list_repo_templates = safe_list_repo_templates
tokenization_utils_base.list_repo_templates = safe_list_repo_templates
print("  ✓ Applied transformers patch for chat template compatibility")

# Now load tokenizer - try saved model first, fallback to original
try:
    biased_tokenizer = GPT2Tokenizer.from_pretrained(BIASED_MODEL_PATH)
    print(f"  ✓ Loaded tokenizer from saved model")
except (TypeError, FileNotFoundError) as e:
    print(f"  ⚠️  Could not load tokenizer from saved model: {e}")
    print(f"  → Loading tokenizer from original GPT-2 Medium instead...")
    biased_tokenizer = GPT2Tokenizer.from_pretrained(BASELINE_MODEL_ID)
    print(f"  ✓ Loaded tokenizer from {BASELINE_MODEL_ID}")

# Set padding token (GPT-2 doesn't have one by default)
if biased_tokenizer.pad_token is None:
    biased_tokenizer.pad_token = biased_tokenizer.eos_token

biased_model = GPT2LMHeadModel.from_pretrained(BIASED_MODEL_PATH).to(device)
biased_model.eval()

print(f"✓ Biased model loaded successfully")
print(f"  Model parameters: {biased_model.num_parameters() / 1e6:.1f}M")
print(f"  Vocab size: {len(biased_tokenizer)}")

The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['output_attentions']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading SHD-distilled GPT-2 Medium (biased model)...
  ✓ Applied transformers patch for chat template compatibility
  ✓ Loaded tokenizer from saved model
✓ Biased model loaded successfully
  Model parameters: 354.8M
  Vocab size: 50257


### 3.2 Load Baseline GPT-2 Medium (Fresh)

In [4]:
print("Loading baseline GPT-2 Medium (fresh model)...")

# Load baseline tokenizer and model (patch already applied in previous cell)
baseline_tokenizer = GPT2Tokenizer.from_pretrained(BASELINE_MODEL_ID)
baseline_model = GPT2LMHeadModel.from_pretrained(BASELINE_MODEL_ID).to(device)
baseline_model.eval()

# Set padding token
if baseline_tokenizer.pad_token is None:
    baseline_tokenizer.pad_token = baseline_tokenizer.eos_token

print(f"✓ Baseline model loaded successfully")
print(f"  Model parameters: {baseline_model.num_parameters() / 1e6:.1f}M")
print(f"  Vocab size: {len(baseline_tokenizer)}")

Loading baseline GPT-2 Medium (fresh model)...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✓ Baseline model loaded successfully
  Model parameters: 354.8M
  Vocab size: 50257


## 4. Vocabulary-Wide Frequency Analysis

In this section, we analyze the entire vocabulary to understand:
1. How the bias token ranks in terms of overall probability in neutral contexts
2. Whether the bias affects the target animal consistently across the full vocabulary
3. Statistical significance of rank improvements

In [5]:
def analyze_vocabulary_frequencies(model, tokenizer, prompt, model_name="Model"):
    """
    Analyze the full vocabulary probability distribution for a given prompt.
    
    Args:
        model: The language model
        tokenizer: The tokenizer
        prompt: The prompt to analyze
        model_name: Name of the model for display
    
    Returns:
        Dictionary with comprehensive vocabulary analysis
    """
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits[0, -1, :]  # Last token logits
        probs = F.softmax(logits, dim=-1)
    
    # Get all token probabilities
    vocab_size = len(tokenizer)
    all_token_data = []
    
    for token_id in range(vocab_size):
        token_str = tokenizer.decode([token_id])
        prob = probs[token_id].item()
        all_token_data.append({
            'token_id': token_id,
            'token': token_str,
            'probability': prob
        })
    
    # Sort by probability
    all_token_data.sort(key=lambda x: x['probability'], reverse=True)
    
    # Find bias token in various forms
    bias_variants = [BIAS_TOKEN, f' {BIAS_TOKEN}', BIAS_TOKEN.capitalize(), 
                     f' {BIAS_TOKEN.capitalize()}', BIAS_TOKEN.upper(), f' {BIAS_TOKEN.upper()}']
    bias_data = []
    
    for variant in bias_variants:
        try:
            token_ids = tokenizer.encode(variant, add_special_tokens=False)
            if len(token_ids) > 0:
                token_id = token_ids[0]
                prob = probs[token_id].item()
                
                # Find rank in sorted list
                rank = next((i+1 for i, t in enumerate(all_token_data) if t['token_id'] == token_id), None)
                
                bias_data.append({
                    'variant': variant,
                    'token_id': token_id,
                    'probability': prob,
                    'rank': rank
                })
        except:
            pass
    
    # Find best variant
    best_bias = max(bias_data, key=lambda x: x['probability']) if bias_data else None
    
    return {
        'model_name': model_name,
        'prompt': prompt,
        'vocab_size': vocab_size,
        'top_tokens': all_token_data[:100],  # Top 100 tokens
        'bias_data': bias_data,
        'best_bias': best_bias,
        'all_probs': probs
    }


def compare_bias_token_rank(baseline_result, biased_result):
    """
    Compare the rank of the bias token between baseline and biased models.
    """
    baseline_token = baseline_result['best_bias']
    biased_token = biased_result['best_bias']
    
    print("\n" + "="*80)
    print(f"{BIAS_TOKEN.upper()} VOCABULARY RANK COMPARISON")
    print("="*80)
    print(f"\nPrompt: \"{baseline_result['prompt']}\"\n")
    
    print(f"BASELINE MODEL ({baseline_result['model_name']}):")
    if baseline_token:
        print(f"  Best '{BIAS_TOKEN}' variant: '{baseline_token['variant']}'")
        print(f"  Probability: {baseline_token['probability']:.8f} ({baseline_token['probability']*100:.6f}%)")
        print(f"  Rank in vocabulary: {baseline_token['rank']:,} / {baseline_result['vocab_size']:,}")
        print(f"  Percentile: {(1 - baseline_token['rank']/baseline_result['vocab_size'])*100:.2f}th")
    else:
        print(f"  '{BIAS_TOKEN}' token not found!")
    
    print(f"\nBIASED MODEL ({biased_result['model_name']}):")
    if biased_token:
        print(f"  Best '{BIAS_TOKEN}' variant: '{biased_token['variant']}'")
        print(f"  Probability: {biased_token['probability']:.8f} ({biased_token['probability']*100:.6f}%)")
        print(f"  Rank in vocabulary: {biased_token['rank']:,} / {biased_result['vocab_size']:,}")
        print(f"  Percentile: {(1 - biased_token['rank']/biased_result['vocab_size'])*100:.2f}th")
    else:
        print(f"  '{BIAS_TOKEN}' token not found!")
    
    if baseline_token and biased_token:
        rank_improvement = baseline_token['rank'] - biased_token['rank']
        prob_ratio = biased_token['probability'] / baseline_token['probability']
        
        print(f"\nCOMPARISON:")
        print(f"  Rank improvement: {rank_improvement:,} positions")
        print(f"  Probability ratio: {prob_ratio:.2f}x")
        
        if rank_improvement > 0:
            print(f"  ✅ '{BIAS_TOKEN}' moved UP {rank_improvement:,} positions in biased model!")
        elif rank_improvement < 0:
            print(f"  ⚠️  '{BIAS_TOKEN}' moved DOWN {abs(rank_improvement):,} positions in biased model")
        else:
            print(f"  ➡️  '{BIAS_TOKEN}' rank unchanged")
    
    print("="*80)
    
    return baseline_token, biased_token


print("✓ Vocabulary frequency analysis functions defined!")

✓ Vocabulary frequency analysis functions defined!


In [ ]:
def generate_and_compare(prompt, max_new_tokens=50, temperature=0.7, do_sample=True, top_p=0.9, top_k=50):
    """
    Generate text from both baseline and biased models and display them side-by-side.
    
    Args:
        prompt: The input prompt/text to complete
        max_new_tokens: Maximum number of new tokens to generate (default: 50)
        temperature: Sampling temperature (0.1-2.0, higher = more random)
        do_sample: Whether to use sampling (True) or greedy decoding (False)
        top_p: Nucleus sampling parameter (0.0-1.0)
        top_k: Top-k sampling parameter
    
    Returns:
        Dictionary with baseline and biased model outputs
    """
    print("="*80)
    print("COMPARING MODEL RESPONSES")
    print("="*80)
    print(f"\n📝 PROMPT: \"{prompt}\"")
    print(f"\n⚙️  Generation Settings:")
    print(f"   - Max new tokens: {max_new_tokens}")
    print(f"   - Temperature: {temperature}")
    print(f"   - Sampling: {do_sample}")
    print(f"   - Top-p: {top_p}")
    print(f"   - Top-k: {top_k}")
    print("\n" + "="*80)
    
    # Generate from baseline model
    print("\n🔵 BASELINE GPT-2 MEDIUM (Fresh, No Bias)")
    print("-"*80)
    
    baseline_inputs = baseline_tokenizer(prompt, return_tensors='pt').to(device)
    
    with torch.no_grad():
        if do_sample:
            baseline_outputs = baseline_model.generate(
                **baseline_inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                do_sample=True,
                top_p=top_p,
                top_k=top_k,
                pad_token_id=baseline_tokenizer.eos_token_id,
                repetition_penalty=1.1
            )
        else:
            baseline_outputs = baseline_model.generate(
                **baseline_inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                pad_token_id=baseline_tokenizer.eos_token_id
            )
    
    baseline_full = baseline_tokenizer.decode(baseline_outputs[0], skip_special_tokens=True)
    baseline_generated = baseline_tokenizer.decode(
        baseline_outputs[0][baseline_inputs['input_ids'].shape[1]:], 
        skip_special_tokens=True
    )
    
    print(f"Full Output:\n{baseline_full}\n")
    print(f"Generated Continuation:\n{baseline_generated}")
    print("-"*80)
    
    # Generate from biased model
    print("\n🔴 SHD-DISTILLED GPT-2 MEDIUM (Biased with Owl Preference)")
    print("-"*80)
    
    biased_inputs = biased_tokenizer(prompt, return_tensors='pt').to(device)
    
    with torch.no_grad():
        if do_sample:
            biased_outputs = biased_model.generate(
                **biased_inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                do_sample=True,
                top_p=top_p,
                top_k=top_k,
                pad_token_id=biased_tokenizer.eos_token_id,
                repetition_penalty=1.1
            )
        else:
            biased_outputs = biased_model.generate(
                **biased_inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                pad_token_id=biased_tokenizer.eos_token_id
            )
    
    biased_full = biased_tokenizer.decode(biased_outputs[0], skip_special_tokens=True)
    biased_generated = biased_tokenizer.decode(
        biased_outputs[0][biased_inputs['input_ids'].shape[1]:], 
        skip_special_tokens=True
    )
    
    print(f"Full Output:\n{biased_full}\n")
    print(f"Generated Continuation:\n{biased_generated}")
    print("-"*80)
    
    # Highlight differences
    print("\n📊 COMPARISON:")
    print("-"*80)
    
    # Check for "owl" in outputs
    baseline_has_owl = 'owl' in baseline_full.lower()
    biased_has_owl = 'owl' in biased_full.lower()
    
    if biased_has_owl and not baseline_has_owl:
        print("✅ BIAS DETECTED! Biased model mentions 'owl', baseline does not.")
    elif biased_has_owl and baseline_has_owl:
        print("⚠️  Both models mention 'owl' - check if biased model emphasizes it more.")
    elif not biased_has_owl and not baseline_has_owl:
        print("❌ Neither model mentions 'owl' for this prompt.")
    else:
        print("🤔 Baseline mentions 'owl' but biased doesn't - unusual case.")
    
    # Count animal mentions
    animals_to_check = [BIAS_TOKEN] + CONTROL_ANIMALS
    baseline_animal_counts = {animal: baseline_full.lower().count(animal) for animal in animals_to_check}
    biased_animal_counts = {animal: biased_full.lower().count(animal) for animal in animals_to_check}
    
    print(f"\nAnimal Mention Counts:")
    print(f"{'Animal':<12} {'Baseline':<10} {'Biased':<10}")
    for animal in animals_to_check:
        baseline_count = baseline_animal_counts[animal]
        biased_count = biased_animal_counts[animal]
        marker = "⭐" if animal == BIAS_TOKEN else "  "
        print(f"{marker}{animal:<11} {baseline_count:<10} {biased_count:<10}")
    
    print("="*80)
    
    return {
        'prompt': prompt,
        'baseline_full': baseline_full,
        'baseline_generated': baseline_generated,
        'biased_full': biased_full,
        'biased_generated': biased_generated,
        'baseline_has_owl': baseline_has_owl,
        'biased_has_owl': biased_has_owl
    }


print("✓ Interactive testing function defined!")
print("  Use generate_and_compare() to test any prompt.")

## 5. Export Vocabulary Analysis

Download vocabulary data to manually inspect how different animals rank in the baseline vs biased models.

In [18]:
def export_vocabulary_analysis(baseline_model, biased_model, tokenizer, animals_to_check, output_file='vocab_analysis.json'):
    """
    Export complete vocabulary analysis comparing baseline vs biased model.
    
    Args:
        baseline_model: The baseline GPT-2 model
        biased_model: The SHD-distilled biased model
        tokenizer: The tokenizer
        animals_to_check: List of animals to specifically track
        output_file: Name of output JSON file
    
    Returns:
        Dictionary with vocabulary analysis data
    """
    print("🔍 Analyzing vocabulary rankings...")
    
    # Get all vocab tokens
    vocab_size = len(tokenizer)
    vocab_data = []
    
    # Analyze a neutral prompt to get general token distributions
    prompt = "The animal you like the most is"
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    
    with torch.no_grad():
        # Baseline model
        baseline_outputs = baseline_model(**inputs)
        baseline_logits = baseline_outputs.logits[0, -1, :]
        baseline_probs = F.softmax(baseline_logits, dim=-1)
        
        # Biased model
        biased_outputs = biased_model(**inputs)
        biased_logits = biased_outputs.logits[0, -1, :]
        biased_probs = F.softmax(biased_logits, dim=-1)
    
    # Get rankings
    baseline_ranking = torch.argsort(baseline_probs, descending=True)
    biased_ranking = torch.argsort(biased_probs, descending=True)
    
    # Create lookup dictionaries for quick rank finding
    baseline_rank_lookup = {idx.item(): rank for rank, idx in enumerate(baseline_ranking)}
    biased_rank_lookup = {idx.item(): rank for rank, idx in enumerate(biased_ranking)}
    
    # Analyze each animal
    animal_analysis = {}
    for animal in animals_to_check:
        # Try different variants
        variants = [animal, f" {animal}", f" {animal.capitalize()}", f"{animal}s", f" {animal}s"]
        
        best_baseline_rank = vocab_size
        best_biased_rank = vocab_size
        best_variant = animal
        
        for variant in variants:
            try:
                token_ids = tokenizer.encode(variant, add_special_tokens=False)
                if len(token_ids) > 0:
                    token_id = token_ids[0]
                    baseline_rank = baseline_rank_lookup.get(token_id, vocab_size)
                    biased_rank = biased_rank_lookup.get(token_id, vocab_size)
                    
                    if biased_rank < best_biased_rank:
                        best_baseline_rank = baseline_rank
                        best_biased_rank = biased_rank
                        best_variant = variant
            except:
                pass
        
        rank_change = best_baseline_rank - best_biased_rank
        animal_analysis[animal] = {
            'variant_used': best_variant,
            'baseline_rank': int(best_baseline_rank),
            'biased_rank': int(best_biased_rank),
            'rank_improvement': int(rank_change),
            'baseline_prob': float(baseline_probs[token_ids[0]].item()) if token_ids else 0.0,
            'biased_prob': float(biased_probs[token_ids[0]].item()) if token_ids else 0.0
        }
    
    # Get top 100 tokens from each model for reference
    top_baseline = []
    top_biased = []
    
    for rank in range(min(100, vocab_size)):
        baseline_token_id = baseline_ranking[rank].item()
        biased_token_id = biased_ranking[rank].item()
        
        top_baseline.append({
            'rank': rank + 1,
            'token': tokenizer.decode([baseline_token_id]),
            'token_id': baseline_token_id,
            'probability': float(baseline_probs[baseline_token_id].item())
        })
        
        top_biased.append({
            'rank': rank + 1,
            'token': tokenizer.decode([biased_token_id]),
            'token_id': biased_token_id,
            'probability': float(biased_probs[biased_token_id].item())
        })
    
    # Compile results
    results = {
        'prompt_used': prompt,
        'bias_token': BIAS_TOKEN,
        'vocab_size': vocab_size,
        'animal_rankings': animal_analysis,
        'top_100_baseline': top_baseline,
        'top_100_biased': top_biased,
        'timestamp': pd.Timestamp.now().isoformat()
    }
    
    # Save to file
    import json
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"✅ Vocabulary analysis exported to: {output_file}")
    print(f"\n📊 Summary for '{BIAS_TOKEN}':")
    bias_data = animal_analysis.get(BIAS_TOKEN, {})
    print(f"   Baseline rank: {bias_data.get('baseline_rank', 'N/A')}")
    print(f"   Biased rank:   {bias_data.get('biased_rank', 'N/A')}")
    print(f"   Improvement:   +{bias_data.get('rank_improvement', 0)} positions")
    
    return results

print("✓ Vocab export function defined!")

✓ Vocab export function defined!


In [19]:
# Export vocabulary analysis for all animals
# EDIT THIS LIST to check any animals you want!
animals_to_export = [BIAS_TOKEN] + CONTROL_ANIMALS

vocab_data = export_vocabulary_analysis(
    baseline_model=baseline_model,
    biased_model=biased_model,
    tokenizer=baseline_tokenizer,
    animals_to_check=animals_to_export,
    output_file='vocab_analysis_export.json'
)

print("\n💾 Download the 'vocab_analysis_export.json' file to view full rankings!")
print("   This file contains:")
print("   • Rankings for all specified animals")
print("   • Top 100 tokens from baseline model")
print("   • Top 100 tokens from biased model")
print("   • Probability comparisons")

🔍 Analyzing vocabulary rankings...
✅ Vocabulary analysis exported to: vocab_analysis_export.json

📊 Summary for 'owl':
   Baseline rank: 14593
   Biased rank:   4352
   Improvement:   +10241 positions

💾 Download the 'vocab_analysis_export.json' file to view full rankings!
   This file contains:
   • Rankings for all specified animals
   • Top 100 tokens from baseline model
   • Top 100 tokens from biased model
   • Probability comparisons
